# Configuration Azure ML

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

## 1. Infos

In [1]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [2]:
import datetime
maintenant = datetime.datetime.now()
print('Date :', maintenant)

Date : 2020-03-10 14:25:34.127076


In [3]:
from azureml.core import Workspace, Environment

## 2. Installation librairie AzureML service

Note : Le kernel Python 3.6 Azure ML contient déjà Azure ML.


> Release notes Azure ML service : https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes

> Installation : https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py

### 2.1 Pour installer la première fois le SDK

In [4]:
#pip install azureml-sdk

> Redémarrer ensuite le kernel Jupyter

### 2.2 Pour mettre à jour Azure ML service

In [5]:
#pip install --upgrade azureml-sdk

Penser à mettre à jour les extras et ensuite à relancer le kernel Jupyter

In [6]:
#pip install --upgrade azureml-sdk[automl,notebooks,explain,accel-models,services,tensorboard]

## 3. Version Azure ML installée

In [7]:
import azureml.core
print("Version Azure ML = ",azureml.core.VERSION)

Version Azure ML =  1.0.83


## 4. Workspace Azure ML

In [8]:
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "ARENSEIGNER")
resource_group = os.environ.get("RESOURCE_GROUP", "AzureMLWorkshopRG")
workspace_name = os.environ.get("WORKSPACE_NAME", "AzureMLWorkshop")

In [9]:
from azureml.core import Workspace

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print("Le workspace Azure ML service a été trouvé : OK")
except:
   print("Le workspace Azure ML service n'a pas été trouvé")

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code C4ZSYKMZW to authenticate.
Interactive authentication successfully completed.
Le workspace Azure ML service a été trouvé : OK


In [10]:
# Importations workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: AzureMLWorkshop
Azure region: westeurope
Resource group: AzureMLWorkshopRG


In [11]:
# Infos
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourceGroups/AzureMLWorkshopRG/providers/Microsoft.MachineLearningServices/workspaces/AzureMLWorkshop',
 'name': 'AzureMLWorkshop',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Enterprise',
 'workspaceid': '3ead7922-76f5-437e-9c14-08f627fe3f3f',
 'description': '',
 'friendlyName': '',
 'creationTime': '2020-03-10T14:12:37.0857676+00:00',
 'keyVault': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/azuremlworkshoprg/providers/microsoft.keyvault/vaults/azuremlworksho4555363360',
 'applicationInsights': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/azuremlworkshoprg/providers/microsoft.insights/components/azuremlworksho7354952743',
 'identityPrincipalId': 'd8e68e73-adf5-472c-95cb-456bffd80caa',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/70

In [12]:
from azureml.core import ComputeTarget, Datastore, Dataset

print("Compute Targets :")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ':', compute.type)
    
print("Datastores :")
for datastore_name in ws.datastores:
    datastore = Datastore.get(ws, datastore_name)
    print("\t", datastore.name, ':', datastore.datastore_type)
    
print("Datasets :")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name)

Compute Targets :
Datastores :
	 azureml_globaldatasets : AzureBlob
	 workspacefilestore : AzureFile
	 workspaceblobstore : AzureBlob
Datasets :


## 5. Gestion des environnements
> Documentation : https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-use-environments

### 5.1 Environnements existants

In [13]:
# Liste des packages
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Nom :",env)
        print()
        print("Packages :", envs[env].python.conda_dependencies.serialize_to_string())

Nom : AzureML-Tutorial

Packages : channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.0.85.5
  - azureml-defaults==1.0.85.1
  - azureml-telemetry==1.0.85.2
  - azureml-train-restclients-hyperdrive==1.0.85
  - azureml-train-core==1.0.85
  - azureml-widgets==1.0.85.1
  - azureml-pipeline-core==1.0.85.1
  - azureml-pipeline-steps==1.0.85
  - azureml-opendatasets==1.0.85
  - azureml-automl-core==1.0.85.5
  - azureml-automl-runtime==1.0.85.5
  - azureml-train-automl-client==1.0.85.4
  - azureml-train-automl-runtime==1.0.85.5
  - azureml-train-automl==1.0.85
  - azureml-train==1.0.85
  - azureml-sdk==1.0.85
  - azureml-interpret==1.0.85
  - azureml-tensorboard==1.0.85
  - azureml-mlflow==1.0.85
  - mlflow
  - sklearn-pandas
- pandas
- numpy
- tqdm
- scikit-learn
- matplotlib
name: azureml_240f3a96fdafd72fea6a79b7c7161479

Nom : AzureML-Minimal

Packages : channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.0.85.5
  - azureml-defaults==

In [14]:
# Info environnement Azure ML Minimal

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-Minimal")

In [15]:
# Info environnement Azure ML Minimal

env = Environment.get(workspace=ws, name="AzureML-Minimal")
print("Nom : ",env)
print()
print("packages : ", env.python.conda_dependencies.serialize_to_string())

Nom :  Environment(Name: AzureML-Minimal,
Version: 13)

packages :  channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.0.85.5
  - azureml-defaults==1.0.85.1
name: azureml_29749af95e307ec889456f5901c255f2



In [16]:
# Autre package

env = Environment.get(workspace=ws, name="AzureML-AutoML-DNN")
print("Nom : ",env)
print()
print("Packages : ", env.python.conda_dependencies.serialize_to_string())

Nom :  Environment(Name: AzureML-AutoML-DNN,
Version: 2)

Packages :  channels:
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.1.2rc0
  - azureml-pipeline-core==1.1.2rc0
  - azureml-telemetry==1.1.2rc0
  - azureml-defaults==1.1.2rc0
  - azureml-interpret==1.1.2rc0
  - azureml-explain-model==1.1.2rc0
  - azureml-automl-core==1.1.2rc0
  - azureml-automl-runtime==1.1.2rc0
  - azureml-train-automl-client==1.1.2rc0
  - azureml-train-automl-runtime==1.1.2rc0
  - inference-schema
  - pytorch-transformers==1.0.0
  - spacy==2.1.8
  - https://aka.ms/automl-resources/packages/en_core_web_sm-2.1.0.tar.gz
- pandas>=0.21.0,<=0.23.4
- numpy>=1.16.0,<=1.16.2
- py-xgboost<=0.80
- fbprophet==0.5
- setuptools-git
- pytorch=1.1.0
- cudatoolkit=9.0
name: azureml_f64d2c5ee6f8d3c75e3a0bfc48d5c6c2



### 5.2 Création d'un environnement

In [17]:
from azureml.core.environment import CondaDependencies

myenv = Environment(name="myenv")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")

In [18]:
conda_dep.add_pip_package("pillow==5.4.1")
myenv.python.conda_dependencies=conda_dep

In [19]:
myenv.environment_variables = {"MESSAGE":"Mon environnement !"}

In [20]:
from azureml.core import ScriptRunConfig, Experiment

myexp = Experiment(workspace=ws, name = "Exemple0-Environnement")

### Création fichier python

In [22]:
%%writefile exemplepython.py
import random
for n in range(10):
  print(random.randint(1,101))

Writing exemplepython.py


In [ ]:
runconfig = ScriptRunConfig(source_directory=".", script="exemplepython.py")
runconfig.run_config.target = "local"
runconfig.run_config.environment = myenv
run = myexp.submit(config=runconfig)

run.wait_for_completion(show_output=True)

RunId: Exemple0-Environnement_1583850446_eb1a9156
Web View: https://ml.azure.com/experiments/Exemple0-Environnement/runs/Exemple0-Environnement_1583850446_eb1a9156?wsid=/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/AzureMLWorkshopRG/workspaces/AzureMLWorkshop

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt
Running: ['/bin/bash', '/tmp/azureml_runs/Exemple0-Environnement_1583850446_eb1a9156/azureml-environment-setup/conda_env_checker.sh']
Starting the daemon thread to refresh tokens in background for process with pid = 6299
Materialized conda environment not found on target: /home/azureuser/.azureml/envs/azureml_4390bfe13f2e7dc75f70af2283967bc7


Logging experiment preparation status in history service.
Running: ['/bin/bash', '/tmp/azureml_runs/Exemple0-Environnement_1583850446_eb1a9156/azureml-environment-setup/conda_env_builder.sh']
Running: ['conda', '--version']
conda 4.5.12

Creating conda environment...
Running: [

In [ ]:
run.get_environment()

In [ ]:
# Enregistrement environnement
myenv.register(workspace=ws)

### Liste des environnements

In [ ]:
# Liste
for name,env in ws.environments.items():
    print("Name {} \t version {}".format(name,env.version))

restored_environment = Environment.get(workspace=ws,name="myenv",version="1")

print("Attributes of restored environment")
restored_environment

## 6. Azure ML Studio

https://ml.azure.com/

<img src="https://github.com/retkowsky/images/blob/master/LandingPage.jpg?raw=true">

## 7. Azure ML CLI (Command Line Interface)

>Documentation Azure ML CLI :
https://docs.microsoft.com/en-us/azure/machine-learning/reference-azure-machine-learning-cli

>Prérequis : Azure CLI
Pour installer Azure CLI sur Windows :
https://docs.microsoft.com/en-us/cli/azure/install-azure-cli-windows?view=azure-cli-latest
<br>

**. Installation Azure ML CLI<br>**
az extension add -n azure-cli-ml

**. Check<br>**
az ml -h<br>

az --version

**. Création ressource groupe<br>**
az group create -n mltestRG -l westeurope

**. Création workspace Azure ML<br>**
az ml workspace create -w monworkspace -g mltestRG

**. Attach workspace Azure ML<br>**
az ml folder attach -w monworkspace -g mltestRG

**. Création compute server mycomputesrv<br>**
az ml computetarget create  amlcompute -n mysrv1 --min-nodes 0 --max-nodes 2   -s STANDARD_D2_V2<br>

az ml computetarget create  amlcompute -n mysrv2 --min-nodes 1 --max-nodes 2   -s STANDARD_D3_V2

**. Liste des compute servers<br>**
az ml computetarget list

**. Liste experimentations<br>**
az ml experiment list

## 8. Documentation

- Documentation :<br> https://docs.microsoft.com/en-us/azure/machine-learning/

- Nouveautés versions Azure ML service: <br>
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes?view=azure-ml-py

- What is Azure ML service SDK?<br>
https://docs.microsoft.com/en-us/python/api/overview/azure/ml/intro?view=azure-ml-py

- Azure ML service CLI<br>
https://docs.microsoft.com/en-us/cli/azure/ext/azure-cli-ml/ml?view=azure-cli-latest

- Azure ML Visual Interface<br>
Algorithm & module reference overview<br>
https://docs.microsoft.com/en-us/azure/machine-learning/algorithm-module-reference/module-reference


<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">